# Филтрирање на слика во фреквенциски домен

## Теорија

За да филтрираме слика во фреквенциски домен 
потребно е да ја помножиме сликата во фреквенциски 
домен со филтерот (кернел) кој е исто така во фреквенциски домен.

$$I_{f,filt} = HI_f$$

За таа цел потребно е да се вметне Octave функција која ќе креира “Band pass gaussian filter” во фреквенциски домен. Bandpass гаусовиот филтер во фреквенциски домен е дефиниран со равенката:


$$H(i,j)=e^{-\dfrac{d(i,j)^2}{f_{h}^{2}}}\left(1-e^{-\dfrac{d(i,j)^2}{f_{l}^{2}}}\right)$$

Каде што $d(i,j)$ е евклидовото растојанието на даден пиксел од центарот на сликата, додека $f_h$ и $f_l$ се праговите на високите и ниските фреквенции кои сакаме да ги филтрираме.
Конверзија на 2D сликата од просторен во фреквенциски домен се прави со користење на Фуриева трансформација. Фуриевата трансформација е дефинирана како:

$$F(u,v)=\underset{-\infty}\iint^{\infty}f(x,y)e^{-i2\pi(ux+vy)}dxdy$$

А во дискретниот домен горната равенка може да биде преведена како

$$F(u,v) = \displaystyle\sum_{m=-\infty}^{\infty} \sum_{n=-\infty}^{\infty} f[m,n] \cdot e^ {-i2\pi(umx_0+vny_0)}$$
        
Конверзија на 2D сликата од фреквенциски во просторен домен се прави со
користење на инверзна Фуриева трансформација. Инверзната Фуриева
трансформација е дефинирана како:

$$f(x,y)=\underset{-\infty}\iint^{\infty}F(u,v)e^{i2\pi(ux+vy)}dudv$$

А во дискретниот домен горната равенка може да биде преведена како

$$F(x,y) = \displaystyle\sum_{m=-\infty}^{\infty} \sum_{n=-\infty}^{\infty} f[m,n] \cdot e^ {i2\pi(xmu_0+ynv_0)}$$

## Имплементација и визуелизација

In [ ]:
%%file create_filter.m

function filter = create_filter(nx,ny,d0,d1)
    filter = ones(nx,ny);
    for i = 0:nx-1
        for j = 0:ny-1
            dist= sqrt((i-nx/2)^2 + (j-ny/2)^2);
            filter(i+1,j+1) = exp(-(dist^2)/(d1^2)).*(1.0-exp(-(dist^2)/(d0^2)));
        end
    end
    filter(nx/2+1,ny/2+1)=1;
end

In [ ]:
clear all
%read image
im = double(imread('Requirements/Barbara.tif'));
%size of the image
[nx ny] = size(im);
%Transform the image from spatial to frequency domain
im_fft = fftshift(fft2(im));
% Create Gaussian bandpass kernels, in frequency domain, for 20 different cut off frequencies and filter the image
p=1;
for i = 1:10:101
q=1;
    for j = 1:10:101
        %Create the bandpass kernel
        tmp1 = create_filter(nx,ny,i,j);
        %Filter the image with the bandpass kernel and transform it back to spatial domain
        tmp = mat2gray(abs(ifft2(ifftshift(tmp1.*im_fft))));
        %Save the filtered image (in spatial domain) and kernel (in frequency domain) for visualization 
        filtered_image(p,q,:) = tmp(:);
        filter_kernel(p,q,:) = tmp1(:);
        q=q+1;
    end
    p=p+1;
end

%Used to save the filtered images and the filtered kernel in separate files
save filtered_image
save filter_kernel

In [ ]:
#Using inline to avoid flickering of the image
%matplotlib inline

import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg
import scipy.io as sio

#Create RangeSlider widget
value = widgets.IntRangeSlider(
    value=[0, 10],
    min=0,
    max=10,
    step=1,
    description='Frequency cutoffs:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout={'width': '100%', 'height':'100%'}
)

#Function to show image according to the slider value
#First value is the low frequency and the second is the highest frequency
def show_image(value_0, value_1):
    filtered_image = sio.loadmat('filtered_image.mat')
    filt_images = filtered_image['filtered_image']
    filt_image = np.rot90((filt_images[value_0][value_1][0:].reshape(402,566)), -1)
    return filt_image;
    
#Function to show kernel image according to the slider value  
#First value is the low frequency and the second is the highest frequency
def show_kernel(value_0, value_1):
    filtered_kernel = sio.loadmat('filter_kernel.mat')
    filt_kernels = filtered_kernel['filter_kernel']
    filt_kernel = np.rot90((filt_kernels[value_0][value_1][0:].reshape(402,566)), -1)
    return filt_kernel;

#Function to show both images side by side
def update_plot(value):
    image = show_image(value[0], value[1])
    kernel = show_kernel(value[0], value[1])
    plot_image = np.concatenate((np.fliplr(image), np.fliplr(kernel)), axis=1)
    plt.figure(figsize=(20,30))
    plt.imshow(plot_image, cmap="gray")
    plt.title('Filtered Image\n')
    plt.axis('off')
    plt.show()
    
widgets.interactive(update_plot, value=value)

# Додатни функционалности

## Селектирање на слика

In [ ]:
import traitlets
from ipywidgets import widgets
from IPython.display import display
from tkinter import Tk, filedialog


class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        with out:
            try:
                # Create Tk root
                root = Tk()
                # Hide the main window
                root.withdraw()
                # Raise the root to the top of all windows.
                root.call('wm', 'attributes', '.', '-topmost', True)
                # List of selected fileswill be set to b.value
                b.files = filedialog.askopenfilename(multiple=True)

                b.description = "Files Selected"
                b.icon = "check-square-o"
                b.style.button_color = "lightgreen"
            except:
                pass
out = widgets.Output()
raw = SelectFilesButton()
widgets.VBox([raw, out])